<a href="https://colab.research.google.com/github/cwinsor/medical_image_uw_madison/blob/main/colab_02_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys
import os

In [3]:
assert ('google.colab' in sys.modules), "ERROR - the script expects to be run in Colab"

In [4]:
# we persist code, dataset and runs on google drive...
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [35]:
work_folder = '/content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison/'
os.chdir(work_folder)
print("checking git status (git fetch --dry-run):")
!pwd
!git fetch --dry-run

checking git status (git fetch --dry-run):
/content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 589 bytes | 15.00 KiB/s, done.
From https://github.com/cwinsor/medical_image_uw_madison
   51d8273..530ff1e  main       -> origin/main


## Comparing ***os.getenv('PATH')*** to ***sys.path***
* the former is path for the environment shell (applications)
* the latter is the python search path for modules. Importantly it is constructed when Python *IS STARTED* and is:
  * the folder of the python interpreter
  * installation-dependent standard folders
  * the contents of PYTHONPATH
  
in other words... updating PYTHONPATH won't help inside a jupyter notebook!


In [21]:
paths = os.getenv('PATH')
for p in paths.split(":"):
    print(p)

/opt/bin
/usr/local/nvidia/bin
/usr/local/cuda/bin
/usr/local/sbin
/usr/local/bin
/usr/sbin
/usr/bin
/sbin
/bin
/tools/node/bin
/tools/google-cloud-sdk/bin


In [30]:
for p in sys.path:
    print(p)

/content
/env/python
/usr/lib/python310.zip
/usr/lib/python3.10
/usr/lib/python3.10/lib-dynload

/usr/local/lib/python3.10/dist-packages
/usr/lib/python3/dist-packages
/usr/local/lib/python3.10/dist-packages/IPython/extensions
/root/.ipython
foo


In [29]:
THE_FOLDER = "foo"
# THE_FOLDER = os.getcwd()

if THE_FOLDER in sys.path:
    print("already exists")
else:
    sys.path.append(THE_FOLDER)
    print("added")

already exists


# PyTorch Distributed...

https://pytorch.org/docs/stable/distributed.html

    How to use this module:

    Single-Node multi-process distributed training

    python -m torch.distributed.launch --nproc-per-node=NUM_GPUS_YOU_HAVE
            YOUR_TRAINING_SCRIPT.py (--arg1 --arg2 --arg3 and all other
            arguments of your training script)

    Multi-Node multi-process distributed training: (e.g. two nodes)

    Node 1: (IP: 192.168.1.1, and has a free port: 1234)
    python -m torch.distributed.launch --nproc-per-node=NUM_GPUS_YOU_HAVE
            --nnodes=2 --node-rank=0 --master-addr="192.168.1.1"
            --master-port=1234 YOUR_TRAINING_SCRIPT.py (--arg1 --arg2 --arg3
            and all other arguments of your training script)

    Node 2:
    python -m torch.distributed.launch --nproc-per-node=NUM_GPUS_YOU_HAVE
            --nnodes=2 --node-rank=1 --master-addr="192.168.1.1"
            --master-port=1234 YOUR_TRAINING_SCRIPT.py (--arg1 --arg2 --arg3
            and all other arguments of your training script)


# Ours is single-node multi-GPU.
So the shell command would be:

    cd /content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison/project/Kaggle-UMWGIT

    CONFIG='work_configs/tract/final_solution/classification_configs/cls_1.py'
    NUM_GPUS_PER_NODE=2
    MASTER_PORT=1234
    ARGS='foo="bar" lala="foofoo"'

    python -m torch.distributed.launch --nproc-per-node=$NUM_GPUS_PER_NODE
            tools/train.py $CONFIG --launcher pytorch $ARGS
            

# let's do that now...

In [ ]:
!pip install mmcv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.5/469.5 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 21.9 MB/s eta 0:00:00


In [40]:
run_folder = '/content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison/project/Kaggle-UWMGIT'
os.chdir(run_folder)
!pwd

/content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison/project/Kaggle-UWMGIT


In [44]:
CONFIG='work_configs/tract/final_solution/classification_configs/cls_1.py'
NUM_GPUS_PER_NODE=1
MASTER_PORT=1234
ARGS='foo="bar" lala="foofoo"'

In [45]:
!python -m torch.distributed.launch --nproc-per-node=$NUM_GPUS_PER_NODE tools/train.py $CONFIG --launcher pytorch $ARGS

/usr/local/lib/python3.10/dist-packages/torch/distributed/launch.py:181: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
Traceback (most recent call last):
  File "/content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison/project/Kaggle-UWMGIT/tools/train.py", line 9, in <module>
    import mmcv
ModuleNotFoundError: No module named 'mmcv'
ERROR:torch.distributed.elastic.multiprocessing.api:failed (exitcode: 1) local_rank: 0 (pid: 57452) of binary: /usr/bin/python3
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/py